## Question 1: **Answer**

In [1]:
import pandas as pd
import sqlite3

In [ ]:
def add_data_to_database(input_data_1, input_data_2):
    try:
        df_1 = pd.read_csv(input_data_1, )
        df_2 = pd.read_csv(input_data_2)
        engine = sqlite3.connect("customer_orders_data.db")
        # Creating orders table
        df_1.to_sql('orders',con=engine, index = False)
        # Creating order items table
        df_2.to_sql('order_items',con=engine, index = False)
        engine.close()
    except Exception as e:
            print(f"Error: {e}")

# Loading data
data_1 = "/content/orders.csv"
data_2 = "/content/order_items.csv"
add_data_to_database(data_1, data_2)

In [3]:
# Loading SQL extension
%load_ext sql

In [4]:
# Connecting to the database
%sql sqlite:///customer_orders_data.db

In [5]:
# Default to duplicated style
%config SqlMagic.style = '_DEPRECATED_DEFAULT'

In [6]:
%%sql
SELECT * FROM orders
LIMIT 5;

 * sqlite:///customer_orders_data.db
Done.


order_id,customer_id,names,restaurant_id,order_date,delivery_fee
1,101,Luca Rossi,202,2024-03-25 12:30:00,5.99
2,102,Park Min woo,201,2024-03-26 18:00:00,3.5
3,103,Akio Suzuki,203,2024-03-28 10:15:00,4.25
4,104,Anna Petrova,204,2021-03-29 10:15:01,4.25
5,105,Alexander Andersen,205,2021-03-30 10:30:02,4.25


In [7]:
%%sql
SELECT * FROM order_items
LIMIT 5;

 * sqlite:///customer_orders_data.db
Done.


order_id,menu_item_id,quantity,item_price
1,11,2,9.99
2,12,1,12.47
3,14,1,18.95
4,15,2,1.42
5,13,3,11.5


## Question 2: **Answer**

In [8]:
%%sql
-- Step 1: Create a view for customer spending
CREATE VIEW IF NOT EXISTS customer_spend AS
SELECT
    o.names,
    oi.quantity,
    oi.item_price,
    o.delivery_fee
FROM orders o
JOIN order_items oi
-- Joining tables on order_id columns
ON o.order_id = oi.order_id;

 * sqlite:///customer_orders_data.db
Done.


[]

In [9]:
%%sql
-- Step 2: Query to calculate Luca Rossi's total bill
SELECT
    names,
    printf('%10s', SUM(quantity * item_price) + delivery_fee)
    AS total_bill
FROM customer_spend
WHERE names = 'Luca Rossi'
GROUP BY names;

 * sqlite:///customer_orders_data.db
Done.


names,total_bill
Luca Rossi,25.97


## Question 3: **Answer**

In [10]:
%%sql
-- Using a CTE to rank the customers by total spend
WITH ranked_spending AS (
    SELECT
        names,
        SUM(quantity * item_price) + delivery_fee AS total_spend,
        RANK() OVER (ORDER BY SUM(quantity * item_price) + delivery_fee ASC)
        AS spending_rank
    FROM customer_spend
    GROUP BY names, delivery_fee
)
-- Select the two lowest-ranked customers (including ties)
SELECT
    names,
    total_spend
FROM ranked_spending
WHERE spending_rank <= 2
ORDER BY total_spend ASC;

 * sqlite:///customer_orders_data.db
Done.


names,total_spend
Yuki Tanaka,4.92
Lars Johansson,6.34


## Question 4: **Answer**

In [11]:
%%sql
-- A view to calculate the total quantities for each menu item
CREATE VIEW IF NOT EXISTS menu_restaurant_quantity AS
SELECT
    order_items.menu_item_id,
    orders.restaurant_id,
    SUM(order_items.quantity) AS total_quantity
FROM order_items
JOIN orders
ON orders.order_id = order_items.order_id
GROUP BY order_items.menu_item_id, orders.restaurant_id;

 * sqlite:///customer_orders_data.db
Done.


[]

In [12]:
%%sql
-- find the menu item with the highest ordered quantity
SELECT
    menu_item_id,
    restaurant_id,
    total_quantity
FROM menu_restaurant_quantity
WHERE total_quantity = (SELECT MAX(total_quantity) FROM menu_restaurant_quantity);

 * sqlite:///customer_orders_data.db
Done.


menu_item_id,restaurant_id,total_quantity
22,212,7


## Question 5: **Answer**

In [13]:
%%sql
-- calculate the total revenue and margin for each menu item
CREATE VIEW IF NOT EXISTS menu_item_margin AS
SELECT
    o.order_date,
    oi.menu_item_id,
    o.restaurant_id,
    SUM(oi.quantity * oi.item_price) AS total_revenue,
    SUM(o.delivery_fee) AS total_delivery_fee,
    SUM(oi.quantity * oi.item_price) - SUM(o.delivery_fee) AS margin
FROM order_items oi
JOIN orders o
    ON oi.order_id = o.order_id
WHERE STRFTIME('%Y', o.order_date) = '2021'
GROUP BY oi.menu_item_id, o.restaurant_id;

-- Returning IDs with the highest margin
SELECT
    STRFTIME('%Y', order_date) AS order_year,
    menu_item_id,
    restaurant_id,
    margin
FROM menu_item_margin
WHERE margin = (SELECT MAX(margin) FROM menu_item_margin);

 * sqlite:///customer_orders_data.db
Done.
Done.


order_year,menu_item_id,restaurant_id,margin
2021,22,212,65.68


In [14]:
# %%sql
# DROP VIEW menu_item_margin;